In [1]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('./data/GoogleNews-vectors-negative300.bin', binary=True)

unable to import 'smart_open.gcs', disabling that module


# 70. 単語ベクトルの和による特徴量

## 後でワンホットベクトルに書き直そうね

In [114]:
import numpy as np
import re
from functools import reduce

In [115]:
def load_data(dir_name, file_name):
    with open(f'{dir_name}{file_name}') as f:
        X = list()
        Y = list()
        for line in f:
            line = line.strip()
            splited_line = line.split('\t')
            X.append(splited_line[0])
            Y.append(splited_line[1])
        return np.asarray(X), np.asarray(Y)

def txt2vec(x):
    vec_x_list = list()
    for txt in x:
        splited_txt = re.split(r'[.,: \']', txt)
        vec_x = np.zeros(300)
        word_count = 0
        for word in splited_txt:
            word = word.strip()
            try:
                vec_x += model[word]
                word_count += 1
            except KeyError:
                pass
        if word_count == 0:
            print(txt)
            continue
        vec_x /= word_count
        vec_x_list.append(vec_x)
    return np.asarray(vec_x_list)

def save_file_npy(dir_name, file_name, x):
    np.save(f'{dir_name}{file_name}', x)
        
def load_file_npy(dir_name, file_name):
    return np.load(f'{dir_name}{file_name}').astype(np.float32)

def chr2num(y):
    converter = dict()
    converter['b'] = 0
    converter['t'] = 1
    converter['e'] = 2
    converter['m'] = 3
    return np.asarray([converter[article_type] for article_type in y])

In [116]:
train_x, train_y = load_data('data/', 'train.txt')
valid_x, valid_y = load_data('data/', 'valid.txt')
test_x, test_y = load_data('data/', 'test.txt')

In [117]:
print(train_x.shape, train_y.shape)
print(valid_x.shape, valid_y.shape)
print(test_x.shape, test_y.shape)

(10680,) (10680,)
(1335,) (1335,)
(1335,) (1335,)


In [118]:
train_x = txt2vec(train_x)
valid_x = txt2vec(valid_x)
test_x = txt2vec(test_x)

In [119]:
train_y = chr2num(train_y)
valid_y = chr2num(valid_y)
test_y = chr2num(test_y)

In [120]:
save_file_npy('work/', 'train_x', train_x)
save_file_npy('work/', 'train_y', train_y)
save_file_npy('work/', 'valid_x', valid_x)
save_file_npy('work/', 'valid_y', valid_y)
save_file_npy('work/', 'test_x', test_x)
save_file_npy('work/', 'test_y', test_y)

# 71. 単層ニューラルネットワークによる予測

In [8]:
train_x = load_file_npy('work/', 'train_x.npy')
train_y = load_file_npy('work/', 'train_y.npy')
valid_x = load_file_npy('work/', 'valid_x.npy')
valid_y = load_file_npy('work/', 'valid_y.npy')
test_x = load_file_npy('work/', 'test_x.npy')
test_y = load_file_npy('work/', 'test_y.npy')

In [48]:
print(train_x.shape, train_y.shape)

(10679, 300) (10680,)


In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

In [10]:
torch.manual_seed(0)
W = Variable(torch.randn(300, 4), requires_grad = True)
print(W)

tensor([[-1.1258, -1.1524, -0.2506, -0.4339],
        [ 0.8487,  0.6920, -0.3160, -2.1152],
        [ 0.3223, -1.2633,  0.3500,  0.3081],
        ...,
        [ 1.1469, -0.1733,  0.0637, -1.2699],
        [-0.6212, -0.2381,  0.0892,  1.8008],
        [-2.0627,  0.3222, -1.1390,  1.2418]], requires_grad=True)


In [11]:
train_x = torch.from_numpy(train_x)
train_y = torch.from_numpy(train_y)

In [12]:
print(F.softmax(torch.matmul(train_x[:1], W), dim = 1))
print(F.softmax(torch.matmul(train_x[:4], W), dim = 1))

tensor([[0.0044, 0.9502, 0.0428, 0.0026]], grad_fn=<SoftmaxBackward>)
tensor([[0.0044, 0.9502, 0.0428, 0.0026],
        [0.2980, 0.6627, 0.0371, 0.0021],
        [0.0586, 0.7506, 0.0133, 0.1775],
        [0.1399, 0.5424, 0.0614, 0.2563]], grad_fn=<SoftmaxBackward>)


# 72. 損失と勾配の計算

http://37ma5ras.blogspot.com/2017/09/pytorch-02.html

In [13]:
criterion = nn.CrossEntropyLoss()
v_train_x = Variable(train_x[:4])
v_train_y = Variable(train_y[:4])

In [14]:
v_train_x = torch.matmul(v_train_x, W)
pred = F.softmax(v_train_x, dim = -1)
print(pred)
loss = criterion(pred, v_train_y.type(torch.long))

tensor([[0.0044, 0.9502, 0.0428, 0.0026],
        [0.2980, 0.6627, 0.0371, 0.0021],
        [0.0586, 0.7506, 0.0133, 0.1775],
        [0.1399, 0.5424, 0.0614, 0.2563]], grad_fn=<SoftmaxBackward>)


In [15]:
print(loss)
loss.backward()
print(W.grad)

tensor(1.5911, grad_fn=<NllLossBackward>)
tensor([[ 2.6124e-03, -1.0254e-03, -1.1798e-03, -4.0727e-04],
        [-3.2793e-03,  2.9144e-03,  3.3780e-04,  2.7053e-05],
        [-6.1659e-03,  2.7133e-03,  1.9983e-03,  1.4543e-03],
        ...,
        [-5.5620e-03,  5.0955e-03,  6.1436e-04, -1.4785e-04],
        [-5.9280e-03,  7.3155e-03, -1.1759e-03, -2.1165e-04],
        [-5.7692e-04, -1.6045e-03,  2.0903e-03,  9.1109e-05]])


In [16]:
print(train_y[:1])
print(F.softmax(torch.matmul(train_x[:1], W), dim = -1))

tensor([3.])
tensor([[0.0044, 0.9502, 0.0428, 0.0026]], grad_fn=<SoftmaxBackward>)


In [17]:
pred = F.softmax(torch.matmul(train_x[:1], W), dim = -1)
print(pred)
loss = criterion(pred, train_y[:1].type(torch.long))

tensor([[0.0044, 0.9502, 0.0428, 0.0026]], grad_fn=<SoftmaxBackward>)


In [18]:
print(loss)
loss.backward()
print(W.grad)

tensor(1.7267, grad_fn=<NllLossBackward>)
tensor([[ 0.0026, -0.0010, -0.0012, -0.0004],
        [-0.0032,  0.0021,  0.0009,  0.0002],
        [-0.0061,  0.0022,  0.0023,  0.0016],
        ...,
        [-0.0056,  0.0050,  0.0007, -0.0001],
        [-0.0061,  0.0096, -0.0028, -0.0007],
        [-0.0005, -0.0020,  0.0024,  0.0002]])


# 73. 確率的勾配降下法による学習

### メモ
```zsh
jupyter labextension install @jupyter-widgets/jupyterlab-manager
```
を実行しようとすると`RuntimeError: npm dependencies failed to install`が出る
```zsh
conda install nodejs
```
でnodejsを再インストールすることで解決

In [142]:
train_x = load_file_npy('work/', 'train_x.npy')
train_y = load_file_npy('work/', 'train_y.npy')
valid_x = load_file_npy('work/', 'valid_x.npy')
valid_y = load_file_npy('work/', 'valid_y.npy')
test_x = load_file_npy('work/', 'test_x.npy')
test_y = load_file_npy('work/', 'test_y.npy')

In [143]:
torch.manual_seed(0)
W = Variable(torch.randn(300, 4), requires_grad = True)
print(W)

tensor([[-1.1258, -1.1524, -0.2506, -0.4339],
        [ 0.8487,  0.6920, -0.3160, -2.1152],
        [ 0.3223, -1.2633,  0.3500,  0.3081],
        ...,
        [ 1.1469, -0.1733,  0.0637, -1.2699],
        [-0.6212, -0.2381,  0.0892,  1.8008],
        [-2.0627,  0.3222, -1.1390,  1.2418]], requires_grad=True)


In [144]:
from tqdm import tqdm_notebook as tqdm

In [145]:
learning_rate = 0.01
ntrain = len(train_x)
#batch_size = 4
nepoch = 100
op = optim.SGD([W], lr = learning_rate)

In [158]:
loss_list = list()
for epoch in tqdm(range(1, nepoch + 1)):
    sum_loss = 0
    for i in range(ntrain):
        batch_x = torch.from_numpy(train_x[i:i+1])
        batch_y = torch.from_numpy(train_y[i:i+1])
        batch_x, batch_y = Variable(batch_x), Variable(batch_y)
        pred = F.softmax(torch.matmul(batch_x, W), dim = -1)
        loss = criterion(pred, batch_y.type(torch.long))
        op.zero_grad()
        loss.backward()
        op.step()
        sum_loss += loss.data.item() * len(batch_x)
    loss_list.append(sum_loss / ntrain)

In [159]:
print(W)

tensor([[-2.0375, -0.7751,  0.5083, -0.6570],
        [ 0.8165,  0.5626, -0.1780, -2.0879],
        [ 0.1719, -0.3804, -0.9507,  0.8744],
        ...,
        [ 1.9262, -0.2389, -1.1102, -0.8110],
        [ 0.5591, -1.5014, -0.2886,  2.2653],
        [-2.6840,  0.1174, -0.7555,  1.6850]], requires_grad=True)


In [160]:
print(loss_list)

[1.2908205520905804, 1.0574674301602867, 1.0079448441050471, 0.9916939019152288, 0.9832527568277795, 0.9778966107060401, 0.9741127230310708, 0.9712556996986215, 0.9689983874001307, 0.9671549549486753, 0.965610970149326, 0.9642915642440096, 0.9631454401192594, 0.9621360724505861, 0.9612366645244623, 0.9604269673259517, 0.959691395122237, 0.9590176842297508, 0.9583959663796068, 0.9578181952517131, 0.9572776550880532, 0.9567686780896526, 0.9562863448195243, 0.9558263134029921, 0.9553846937440308, 0.9549579104066788, 0.9545426289352138, 0.9541356343566701, 0.9537337906798173, 0.9533339049876406, 0.9529326059175341, 0.9525261842356193, 0.9521102910798587, 0.9516796168316616, 0.9512274755688196, 0.9507452228765809, 0.9502215495493528, 0.9496416275532504, 0.948986126856411, 0.9482306069202637, 0.9473461892553483, 0.9463031243295705, 0.9450781335712373, 0.9436637462748124, 0.9420749689755815, 0.9403491972035236, 0.9385403562239494, 0.9367079648185759, 0.9349018144361981, 0.933153022948499, 0.9

# 74. 正解率の計測

In [161]:
from sklearn.metrics import accuracy_score

In [162]:
pred_y = torch.argmax(F.softmax(torch.matmul(torch.from_numpy(test_x), W), dim = -1), dim = 1)
accuracy_score(test_y, pred_y.detach().numpy())

0.8771535580524344

In [163]:
pred_y = torch.argmax(F.softmax(torch.matmul(torch.from_numpy(valid_x), W), dim = -1), dim = 1)
accuracy_score(valid_y, pred_y.detach().numpy())

0.8696629213483146

In [164]:
pred_y = torch.argmax(F.softmax(torch.matmul(torch.from_numpy(train_x), W), dim = -1), dim = 1)
accuracy_score(train_y, pred_y.detach().numpy())

0.8794943820224719

In [165]:
tempW = Variable(torch.randn(300, 4), requires_grad = True)
pred_y = torch.argmax(F.softmax(torch.matmul(torch.from_numpy(train_x), tempW), dim = -1), dim = 1)
accuracy_score(train_y, pred_y.detach().numpy())

0.16310861423220974

# 75. 損失と正解率のプロット

In [166]:
from torch.utils.tensorboard import SummaryWriter

In [172]:
def execution(data_x, data_y, op, criterion, batch_size = 4, nepoch = 100, training_flag = True):
    ntrain = len(data_x)
    perm = np.random.permutation(ntrain)
    sum_loss = 0
    for i in range(0, ntrain, batch_size):
        batch_x = torch.from_numpy(data_x[perm[i:i + batch_size]])
        batch_y = torch.from_numpy(data_y[perm[i:i + batch_size]])
        batch_x, batch_y = Variable(batch_x), Variable(batch_y)
        pred = F.softmax(torch.matmul(batch_x, W), dim = -1)
        loss = criterion(pred, batch_y.type(torch.long))
        if training_flag:
            op.zero_grad()
            loss.backward()
            op.step()
        sum_loss += loss.data.item() * len(batch_x)
    return sum_loss / ntrain

In [178]:
train_loss_list = []
valid_loss_list = []
writer = SummaryWriter(log_dir='./work/logs')

for epoch in tqdm(range(1, nepoch + 1)):
    train_loss = execution(train_x, train_y, op, criterion)
    writer.add_scalar("train_loss", train_loss, i)
    valid_loss = execution(valid_x, valid_y, op, criterion, training_flag = False)
    writer.add_scalar("valid_loss", valid_loss, i)
    #print(f'train_loss:{train_loss}\nvalid_loss:{valid_loss}')
    train_loss_list.append(train_loss)
    valid_loss_list.append(valid_loss)
writer.close()

KeyboardInterrupt: 

# 76. チェックポイント

# 77. ミニバッチ化

In [ ]:
def train(train_x, train_y, batch_size = 4, nepoch = 100, op, criterion):
    ntrain = len(train_x)
    perm = np.random.permutation(ntrain)
    loss_list = list()
    for epoch in tqdm(range(1, nepoch + 1)):
        sum_loss = 0
        for i in range(0, ntrain, batch_size):
            batch_x = torch.from_numpy(train_x[perm[i:i + batch_size]])
            batch_y = torch.from_numpy(train_y[perm[i:i + batch_size]])
            batch_x, batch_y = Variable(batch_x), Variable(batch_y)
            pred = F.softmax(torch.matmul(batch_x, W), dim = -1)
            loss = criterion(pred, batch_y.type(torch.long))
            op.zero_grad()
            loss.backward()
            op.step()
            sum_loss += loss.data.item() * len(batch_x)
        loss_list.append(sum_loss / ntrain)

# 78. GPU上での学習

In [136]:
print(torch.cuda.is_available())

False


# 79. 多層ニューラルネットワーク